<a href="https://colab.research.google.com/github/Sundaynot/HP_Big_data_project/blob/main/HP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive2
#l'output 'xxx is not a symbolic link' non influirà sull'implementazione o esecuzione
#per risolvere 'xxx is not a symbolic link', commenta le righe che iniziano con !mv xxxx
#dovresti sostituire xxx.11 con la versione corretta se si verificano altri errori dopo un aggiornamento di Colab
#per ottenre la versione corretta, usa !ls /usr/local/lib per verificarla
#!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
#!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
#!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
#!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
#!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
#!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
# !sudo ldconfig
#Se si verifica un errore relativo all'esecuzione sopra indicata, puoi provare a commentare le 12 righe sopra sotto l'installazione di
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#l'output 'xxx is not a symbolic link' non influirà sulla tua implementazione o esecuzione
#per risolvere 'xxx is not a symbolic link', puoi commentare le righe che iniziano con !mv xxxx
#dovresti sostituire xxx.11 con la versione corretta se si verificano altri errori dopo un aggiornamento di Colab
#per ottenre la versione corretta, usa !ls /usr/local/lib per verificarla

ln: failed to create symbolic link '/usr/local/lib/libtbbmalloc_proxy.so.2': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbmalloc.so.2': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbbind_2_5.so.3': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbb.so.12': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbbind_2_0.so.3': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbbind.so.3': File exists
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 39.6 MB of archives.
After this operation,

In [2]:
# Creazione della SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("HP_Analysis") \
    .getOrCreate()

print("Sessione creata")

Sessione creata


In [3]:
# Nome della cartella e URL del repo
repo_name = "HP_Big_data_project"
repo_url = "https://github.com/sundaynot/HP_Big_data_project.git"

# Clona il repo solo se non esiste già
if not os.path.exists(repo_name):
    print(f"Clonazione del repository '{repo_name}'...")
    !git clone {repo_url}
else:
    print(f"Repository '{repo_name}' già presente.")

Clonazione del repository 'HP_Big_data_project'...
Cloning into 'HP_Big_data_project'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 26 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 2.24 MiB | 4.26 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [4]:
# importo librerie utili
from pyspark.sql.functions import col, lower, regexp_replace, explode, split, concat_ws, monotonically_increasing_id, lag, when

In [8]:
df_hp1 = spark.read.text("/content/HP_Big_data_project/database/01 Harry Potter and the Sorcerers Stone.txt")
df_hp2 = spark.read.text("/content/HP_Big_data_project/database/02 Harry Potter and the Chamber of Secrets.txt")
df_hp3 = spark.read.text("/content/HP_Big_data_project/database/03 Harry Potter and the Prisoner of Azkaban.txt")
df_hp4 = spark.read.text("/content/HP_Big_data_project/database/04 Harry Potter and the Goblet of Fire.txt")
df_hp5 = spark.read.text("/content/HP_Big_data_project/database/05 Harry Potter and the Order of the Phoenix.txt")
df_hp6 = spark.read.text("/content/HP_Big_data_project/database/06 Harry Potter and the Half-Blood Prince.txt")
df_hp7 = spark.read.text("/content/HP_Big_data_project/database/07 Harry Potter and the Deathly Hallows.txt")

In [10]:
df_hp1.show(1, truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                  |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|M r. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you’d expect to be involved in anything strange or

In [11]:
df_hp2.show(1, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Not for the first time, an argument had broken out over breakfast at number four, Privet Drive. Mr. Vernon Dursley had been woken in the early hours of the morning by a loud, hooting noise from his nephew Harry’s room.|
+-------------------------------------------------------------------------------------------------------------------

In [12]:
df_hp3.show(1, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                            |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Harry Potter was a highly unusual boy in many ways. For one thing, he hated the summer holidays more than any other time of year. For another, he really wanted to do his h

In [13]:
df_hp4.show(1, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                                                                                |
+-----------------------------------------------------------------------------------------------------------------------

In [14]:
df_hp5.show(1, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                            

In [15]:
df_hp6.show(1, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                    

In [16]:
df_hp7.show(1, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------